In [219]:
import torch
from torch  import nn 
import numpy as np
import pandas as pd
import spacy
import unicodedata
import re
import torch.nn.functional as F

## preprocessing

#### reading sentence from language pair file
#### getting senteces and ASCII normalize them(getting rid of awkward letters, and puncts)
#### create pairs of sentences (text, its translation)
#### creation of voca for each lang, w2i , i2w mappers and etc.

In [220]:
df = pd.read_csv('tur.txt',delimiter='\t',header=None)

In [221]:
df.tail(10)

,0,1,2
518116,We need to uphold laws against discrimination ...,"Ayrımcılığa karşı yasaları, işe alma, konut ed...",CC-BY 2.0 (France) Attribution: tatoeba.org #5...
518117,Tom made the comment so offhandedly that at fi...,Tom öyle düşüncesizce yorum yaptı ki başlangıç...,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
518118,"Top-down economics never works, said Obama. ""T...",Obama yukarıdan aşağıya ekonominin asla işe ya...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
518119,A carbon footprint is the amount of carbon dio...,Bir karbon ayakizi bizim faaliyetlerimizin bir...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
518120,"At a moment when our economy is growing, our b...",Ekonomimizin büyüdüğü bir anda bizim işletmele...,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
518121,Using high heat settings while ironing synthet...,Sentetik kumaşları ütülerken yüksek ısı ayarla...,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
518122,"If you want to sound like a native speaker, yo...","Eğer bir yerli gibi konuşmak istiyorsan, banjo...",CC-BY 2.0 (France) Attribution: tatoeba.org #9...
518123,If someone who doesn't know your background sa...,Senin geçmiş deneyimini bilmeyen biri senin bi...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
518124,I know that adding sentences only in your nati...,Sadece kendi ana dilinde ya da en güçlü olduğu...,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
518125,Doubtless there exists in this world precisely...,Kuşkusuz bu dünyada her erkeğin ve kadının evl...,CC-BY 2.0 (France) Attribution: tatoeba.org #7...


In [222]:
df.drop(2,axis=1,inplace = True)
df

,0,1
0,Hi.,Selam.
1,Hi.,Merhaba.
2,Run!,Kaç!
3,Run!,Koş!
4,Run.,Kaç!
...,...,...
518121,Using high heat settings while ironing synthet...,Sentetik kumaşları ütülerken yüksek ısı ayarla...
518122,"If you want to sound like a native speaker, yo...","Eğer bir yerli gibi konuşmak istiyorsan, banjo..."
518123,If someone who doesn't know your background sa...,Senin geçmiş deneyimini bilmeyen biri senin bi...
518124,I know that adding sentences only in your nati...,Sadece kendi ana dilinde ya da en güçlü olduğu...


In [223]:
df.rename({0:'eng',1:'turk'},axis=1,inplace=True)
def clean(x):
    return x.strip()
df.eng = df.eng.apply(clean)
df.turk = df.turk.apply(clean)

#### normalizing the unusual to english characters

In [224]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?ı]+", r" ", s)
    return s

In [225]:
df.turk = df.turk.apply(unicodeToAscii)

0

In [226]:
df.eng = df.eng.apply(normalizeString)
df.turk = df.turk.apply(normalizeString)

In [227]:
df.values

array([['hi .', 'selam .'],
       ['hi .', 'merhaba .'],
       ['run !', 'kac !'],
       ...,
       ['if someone who doesn t know your background says that you sound like a native speaker it means they probably noticed something about your speaking that made them realize you weren t a native speaker . in other words you don t really sound like a native speaker .',
        'senin gecmis deneyimini bilmeyen biri senin bir yerli konusmacı gibi konustugunu soylerse bu senin bir yerli konusmacı olmadıgını onlara fark ettiren konusman hakkında muhtemelen bir sey fark ettigi anlamına gelir .yani senin gercekten yerli konusmacı gibi konusmadıgını .'],
       ['i know that adding sentences only in your native or strongest language is probably not as much fun as practicing writing foreign languages but please don t add sentences to the tatoeba corpus if you are not absolutely sure they are correct . if you want to practice languages that you are studying please do so by using a website desig

#### language class, vocab, w2i, i2w

In [228]:
class Language:
    def __init__(self,name):
        self.name = name
        #initially, we have sos and eos
        self.n_words = 2
        self.index2word = {0:"SOS",1:"EOS"}
        self.word2index = {}
        self.word2count = {}
    
    def addWord(self,word):
        #check if word is not existed in words
        if word not in self.word2index:
            #adds word into w2i
            self.word2index[word] = self.n_words
            #adds index into i2w
            self.index2word[self.n_words] = word
            # increases word_numbers, it s done after, because indexing stars from 0
            self.n_words += 1
            #counts word occurance
            self.word2count[word] = 1
        else:
            #if word exists, it increases number of occurance
            self.word2count[word] += 1

    #takes sentence, splits into words and applies add_word method
    def addSentence(self,sentence):
        for word in sentence.split():
            self.addWord(word)

#### language object, pairs generator

In [229]:
lang1_name = 'eng'
lang2_name = 'turk'


    

In [230]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines


    # Split every line into pairs and normalize
    pairs = df.values

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Language(lang2)
        output_lang = Language(lang1)
    else:
        input_lang = Language(lang1)
        output_lang = Language(lang2)

    return input_lang, output_lang, pairs

In [231]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[0].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [232]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'turk')


Reading lines...
Read 518126 sentence pairs
Trimmed to 27029 sentence pairs
Counting words...
Counted words:
eng 5221
turk 13477


In [233]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [234]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [250]:
EOS_token = 1
SOS_token = 0 
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.strip().split()]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair,input_lang,output_lang):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [236]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [237]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [238]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [252]:
import random
from torch import optim
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs),input_lang,output_lang)
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]

        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [253]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

KeyboardInterrupt: 